# [Exploration 1] 인공지능하고 가위바위보하기

#### 위와 같은 헤더를 가졌지만 실제로 가위바위보를 하는건 아니다.    
#### 사진을 보고 [가위] [바위] [보] 를 맞추는 인공지능 분류기(classifier)를 만들어보겠다.

_* 코드가 어떻게 이루어졌는데 하나하나 분석하지는 않았다. 이번 단계에서는 이미 짜져있는 코드를 활용해서 분류기를 만든다_

---

### (1) 데이터셋 사진 촬영 및 디렉토리 설정하기

먼저, 테스트셋으로 사용할 가위, 바위, 보의 사진 각 100장을 준비한다. 사진은 구글의 teachable machine을 활용하여 **순식간에** 100장을 촬영할 수 있다.   
[Teachable Machine](https://teachablemachine.withgoogle.com/)   
   
사진이 준비 되었다면, 가위, 바위 보 각각의 디렉토리를 만들어 사진을 준비시킨다 (100장 씩!).

*__Tip:__ ```mkdir -p``` __를 사용하면 중간 단계의 디렉토리를 자동적으로 생성해주며 하위 디렉토리까지 생성할 수 있다!__*

```mkdir -p ~/aiffel/가위바위보/가위```   
```mkdir -p ~/aiffel/가위바위보/바위```   
```mkdir -p ~/aiffel/가위바위보/보```   

#(mkdir -p가 쥬피터에서는 실행이 안되어 터미널에서 실행하였음.)

### (2) 이미지 리사이징

다음 스텝으로는 **이미지 리사이징**이 있다.   
teachable machine에서 촬영한 사진은 224x224 픽셀 사이즈로, 분류기를 만들기에 불필요하게 크니   
속도 향상을 위해서 28x28사이즈로 줄여보겠다.

아래의 PIL 라이브러리 패키지를 다운받고 설치해준다.

In [21]:
!pip install pillow   

from PIL import Image
import os, glob
import numpy as np

이후 아래 코드를 실행하여 이미지 크기를 줄여준다 --> 28x28 사이즈로!

In [22]:
import os

#가위, 바위, 보 모두 리사이징을 해준다.
#image_dir_path에서 사진이 들어있는 올바른 위치를 설정해준다.

#1. 가위

image_dir_path = os.getenv("HOME") + "/aiffel/가위바위보/가위"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")
    
    
#2. 바위

image_dir_path = os.getenv("HOME") + "/aiffel/가위바위보/바위"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")
    
#3. 보

image_dir_path = os.getenv("HOME") + "/aiffel/가위바위보/보"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

이미지 디렉토리 경로:  /home/aiffel/aiffel/가위바위보/가위
이미지 디렉토리 경로:  /home/aiffel/aiffel/가위바위보/바위
이미지 디렉토리 경로:  /home/aiffel/aiffel/가위바위보/보


가위, 바위, 보 사진이 각각 28 * 28 사이즈로 리사이징 되어 저장되었다!

### (3) load_data

In [61]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=3600   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/가위/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/바위/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/보/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/가위바위보"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3599 입니다.
x_train shape: (3600, 28, 28, 3)
y_train shape: (3600,)


위 함수를 통해 x_train와 y_train dataset의 값을 입력해 주었다.   
가위, 바위, 보 각각 1200장으로 총 3600장의 사진이 들어갔다.

---

### (3) 딥러닝 네트워크 설계하기

먼저, **텐서플로우(tensorflow)**와 **케라스(Keras)**를 **import**하여 활성화(?) 시켜준다.

In [67]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__) #tenserflow가 제대로 import되었는지 확인하기 위해 버전을 출력해본다....

2.2.0


여기서는 **tensorflow** 케라스의 **Sequential Model**을 사용한다. 아래 코드를 입력하여 **Sequential API**를 사용할 수 있다.   
_(설명에 따르면, Sequential API를 통해 **딥러닝 레이어를 쉽게** 사용할 수 있다고 한다. 하지만 개발의 자유도가 떨어진다고 한다.)_

In [38]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

아래와 같이 ```model.summar()``` 메서드를 통해 네트워크 모델을 확인할 수 있다.

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                25632     
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [62]:
import os
import glob

image_dir_path = os.getenv("HOME") + "/aiffel/가위바위보"
(x_train, y_train)=load_data(image_dir_path)


print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3599 입니다.
x_train shape: (3600, 28, 28, 3)
y_train shape: (3600,)


```.shape```을 통해 학습된 데이터 내용을 확인할 수 있다.   
3600장의 사진은 28x28의 픽셀을 가지고 있고, 3채널의 사진임을 확인할 수 있다.

In [63]:
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test / 255.0

print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape(-1,  28, 28, 3)
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))

#혹시 모를때!!

학습데이터(x_train)의 이미지 개수는 3599 입니다.
Before Reshape - x_train_norm shape: (3600, 28, 28, 3)
Before Reshape - x_test_norm shape: (3600, 28, 28, 3)
After Reshape - x_train_reshaped shape: (3600, 28, 28, 3)
After Reshape - x_test_reshaped shape: (3600, 28, 28, 3)


이 부분은 빼도 되지 않나 싶다. 왜냐아면 이미 3채널로 나와있기 때문에.

### (4) 딥러닝 네트워크 학습시키기 !

In [68]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_reshaped, y_train, epochs=10)

Epoch 1/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1378 - accuracy: 0.9608
Epoch 2/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1084 - accuracy: 0.9728
Epoch 3/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1050 - accuracy: 0.9686
Epoch 4/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0757 - accuracy: 0.9797
Epoch 5/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0605 - accuracy: 0.9839
Epoch 6/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0533 - accuracy: 0.9858
Epoch 7/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0476 - accuracy: 0.9878
Epoch 8/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0429 - accuracy: 0.9892
Epoch 9/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0376 - accuracy: 0.9897
Epoch 10/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0312 - accuracy: 0.9925

학습을 시키니 꽤 높은 정확도를 보인다 ㅎㅎ~!   
프로젝트 제출을 준비하는 과정에서 여기서 알수 없는 오류로 막혔었다.   
슬랙에 질문을 하니 퍼실님들께서 구글링을 통해 찾을 수 있도록 유도를 해주셨고, 찾아보니 위에 layer에 input이 1이 되어 있어 오류가 있었다.   
아래는 스스로 해답을 찾는 과정에서 공부한 내용이다. 결론적으론 이것들이 문제는 아니었다.

```complie()``` 메서드는 모델 학습 과정을 설명해준다. 즉, 모델을 빌드하고 실행하기 전에 컴파일 하는 훈련준비단계라고 생각하면 된다.   


* loss : 최적화 과정에서 최소화될 손실함수를 설정하는 것으로, MSE(평균 제곱 오차)와 binary_crossentropy가 자주 사용된다

* optimizer : 훈련 과정을 설정하는 것으로, Adam, SGD 등이 있다

* metrics : 훈련을 모니터링하기 위해 사용한다

```fit()``` 메서드는 주어진 epoch 수 만큼 모델을 학습시킨다.
* batch_size 는 한 번에 사용해야 하는 트레이닝 데이터의 크기를 지정하는 것
* vervise 는 학습 진행 상황에 대한 출력 여부를 지정하는 것 (0(silent), 1(progress bar), 2(one line per epoch))`

_(위 내용들은 블로그 https://blog.naver.com/handuelly/221822938182 를 참고하였음)_

### (5) 얼마나 잘 만들었는지 확인하기!

In [56]:
def load_test_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300  
    img_size=28
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img   
        labels[idx]=0  
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1 
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img 
        labels[idx]=2  
        idx=idx+1
        
    print("학습데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels
image_dir_path = os.getenv("HOME") + "/aiffel/RCP"
(x_test, y_test)=load_test_data(image_dir_path)
x_test_norm = x_test/255.0

학습데이터(x_test)의 이미지 개수는 300 입니다.


위 ```load_test_data```함수를 통해 테스트 데이터셋을 가져올 수 있다.

### (6)두근두근 테스트 정확도 확인하기!!

In [65]:
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

113/113 - 0s - loss: 103.4845 - accuracy: 0.7897
test_loss: 103.48452758789062 
test_accuracy: 0.789722204208374


**78퍼센트**라는 높은 정확도를 보인다!   
딱히 하이퍼파라미터를 바꿀 필요가 없어졌다.

### 회고:   
1. 이번 프로젝트에서 어려웠던 점,   
처음 하는 Exploration project이다보니, 이제 처음 프로그래밍을 배우는 나한테는 모든 내용이 어렵고 벅찼다.   
X와 Y가 각각 무엇을 의미하는이 이해하는데 조차 한참이 걸렸다.   
그리고 짜여진 코드들을 하나하나 해석해야한다는 데에 부담감을 느꼈다.   
차차 배울 것이고, 공부할 것이고, 익숙해지겠지만, 전반적으로 난이도가 너무 높다고 느껴졌다.   

   
2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.   
매 프로젝트마다 새로운걸 많이 배운다. 그 과정에서 스스로 찾아보고 공부해야할게 많다.   
모호한 점은, 아직 코드를 전체적으로 이해하지 못하고 복사/붙여넣기만 했다는 점이 아쉽다.   
나중에 여력이 생기면 오늘 제출하는 프로젝트 파일을 보고 꼭 이해해보겠다.   


3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.   
처음에 300장의 데이터셋을 가지고 테스트를 했을 때에는 34%정도의 정확도가 나왔다.   
이후 2100장을 가지고 재시도를 했는데 50퍼센트 조금 넘었던 것 같다.   
다른 사람들에게 물어보니 3000장 이상을 테스트하면 좋은 결과가 나온다길래 3600장을 넣었더니 높은 정확도가 나왔다.   
파라미터들을 조금 바꿔볼 수도 있었는데 일단 데이터를 많이 학습시켜야겠다는 생각에 파라미터를 바꿔보지는 못했다.   
   
   
4. 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정.   
파라미터와 학습 데이터가 좋지 못한 이유가 있을것 같다.   
다행이도 나의 경우에는, 파라미터를 바꿔보기 전에 충분한 양의 데이터셋을 넣었더니 높은 정확도가 나왔다.  
   
   
5. 자기 다짐   
Aiffel 과정을 시작한지 2주 정도 되었는데 아직도 부족함을 많이 느낀다.   
기본이 너무 없다보니 따라가는데 속도가 느리고 시간이 많이 걸린다.   
구정을 기준으로 파이썬이라던지, 수학적인 부분에 기본기를 탄탄히 다질 수 있도록 시간투자를 많이 해야겠다.   
   
   